# Aggrégation des fichiers de data.gouv.fr pour les PLF et PLR

In [1]:
import csv
import json
import os

from tqdm import tqdm

## Décodage des fichiers
Réécrire les fichier dans un encoding compatible `UTF-8`.

In [2]:
RAW_FOLDER = './raw_data/plf_2017'
DECODED_FOLDER = './decoded_data/plf_2017'

if not os.path.exists(DECODED_FOLDER):
    os.makedirs(DECODED_FOLDER)

In [3]:
for fileName in os.listdir(RAW_FOLDER):
    os.system('iconv -f ISO-8859-1 -t UTF-8 {} > {}'.format(os.path.join(RAW_FOLDER, fileName), os.path.join(DECODED_FOLDER, fileName)))

## PLF

Pour une année donnée, on récupère les **Autorisations d'engagements** et les **Crédits de Paiement** par `Type de mission` > `Mission` > `Programme` > `Action` > `Sous-action`.

Le but est de générer un fichier avec les colonnes suivantes :<br>
`Type de mission` ; `Code Type de Mission` ; `Mission` ; `Code Mission` ; `Programme` ; `Code Programme` ; `Action` ; `Code Action` ; `Sosu-action` ; `Code sous-action` ; `Autorisations d'engagements` ; `Crédits de paiement`

Voici la liste des `Code Type de mission`, déduits des données récupérées lors du hackathon Datafin :

In [4]:
type_de_mission_to_code = {
    "Budget général": "1",
    "Budgets annexes": "2",
    "Comptes d'affectation spéciale": "3",
    "Comptes de concours financiers": "4"
}

### 2019

In [16]:
SOURCE_FOLDER = './decoded_data/plf_2019'
TARGET_FOLDER = './parsed_data'
TARGET_FILE = 'plf_2019.csv'

type_de_mission_to_filename = {
    "Budget général": "bg",
    "Budgets annexes": "ba",
    "Comptes d'affectation spéciale": "cs",
    "Comptes de concours financiers": "cf"
}

In [17]:
mission_to_code = {}

with open(os.path.join(SOURCE_FOLDER, 'plf2019-nomenclature-mpa.csv'), 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=';')
    # On saute la première ligne parce que c'est un header
    next(csv_reader, None)

    for row in csv_reader:
        code_mission = row[2]
        mission = row[3]

        mission_to_code[mission] = code_mission

In [18]:
credits_paiement_total = 0
autorisations_engagement_total = 0

with open(os.path.join(TARGET_FOLDER, TARGET_FILE), 'w+') as output_file:
    # Écrire les headers
    output_file.write("Type de mission;Code type de mission;Mission;Code mission;Programme;" +\
                      "Code programme;Action;Code action;Sous-action;Code sous-action;" +\
                      "Autorisations d'engagement;Crédits de paiement\n")

    # On ouvre chacun des fichiers...
    for type_de_mission in type_de_mission_to_code:
        code_type_de_mission = type_de_mission_to_code[type_de_mission]
        code_filename = type_de_mission_to_filename[type_de_mission]

        with open(os.path.join(
            SOURCE_FOLDER,
            'plf2019-{}-msn-dest.csv'.format(code_filename)), 'r') as csv_file:
            
            csv_reader = csv.reader(csv_file, delimiter=';')
            # On saute la première ligne parce que c'est un header
            next(csv_reader, None)

            for index, row in enumerate(csv_reader):
                # Petite subtilité pour le cas 2019, certains documents csv ont trop de lignes (il y a des lignes vides à la fin)
                if (row[2] == ''):
                    continue
                    
                autorisations_engagement = int(row[9].replace(' ', '')) if row[9].replace(' ', '') != '' else 0
                autorisations_engagement_total += autorisations_engagement
                credits_paiement = int(row[10].replace(' ', '')) if row[10].replace(' ', '') != '' else 0
                credits_paiement_total += credits_paiement

                # On écrit dans le fichier cible toutes les informations utiles
                output_file.write('{};{};{};{};{};{};{};{};{};{};{};{}\n'.format(
                    type_de_mission,
                    code_type_de_mission,
                    row[2],
                    mission_to_code[row[2]],
                    row[4],
                    row[3],
                    row[6],
                    row[5],
                    row[8],
                    row[7],
                    autorisations_engagement,
                    credits_paiement
                ))

print(credits_paiement_total, autorisations_engagement_total)

### 2018

**Il manque le code mission dans tous les fichiers**, j'ai dû utiliser les nomenclatures que j'avais récupérées pendant le hackathon Datafin.

In [29]:
SOURCE_FOLDER = './decoded_data/plf_2018'
TARGET_FOLDER = './parsed_data'
TARGET_FILE = 'plf_2018_ok.csv'

type_de_mission_to_filename = {
    "Budget général": "BG",
    "Budgets annexes": "BA",
    "Comptes d'affectation spéciale": "CS",
    "Comptes de concours financiers": "CF"
}

In [30]:
mission_to_code = {}
code_to_programme = {}

with open(os.path.join('./raw_data/nomenclatures_plf/2018.csv'), 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    # On saute la première ligne parce que c'est un header
    next(csv_reader, None)

    for row in csv_reader:
        code_mission = row[3]
        mission = row[4]
        code_programme = row[5]
        programme = row[6]

        mission_to_code[mission] = code_mission
        code_to_programme[code_programme] = programme

In [33]:
credits_paiement_total = 0
autorisations_engagement_total = 0

with open(os.path.join(TARGET_FOLDER, TARGET_FILE), 'w+') as output_file:
    # Écrire les headers
    output_file.write("Type de mission;Code type de mission;Mission;Code mission;Programme;" +\
                      "Code programme;Action;Code action;Sous-action;Code sous-action;" +\
                      "Autorisations d'engagement;Crédits de paiement\n")

    # On ouvre chacun des fichiers...
    for type_de_mission in type_de_mission_to_code:
        code_type_de_mission = type_de_mission_to_code[type_de_mission]
        code_filename = type_de_mission_to_filename[type_de_mission]

        with open(os.path.join(
            SOURCE_FOLDER,
            'PLF2018-{}-Msn-Dest.csv'.format(code_filename)), 'r') as csv_file:
            
            csv_reader = csv.reader(csv_file, delimiter=';')
            # On saute les headers (4 lignes)
            for i in range(4):
                next(csv_reader, None)

            for index, row in enumerate(csv_reader):
                # Petite subtilité pour le cas 2019, certains documents csv ont trop de lignes (il y a des lignes vides à la fin)
                if (row[2] == ''):
                    continue
                    
                autorisations_engagement = int(row[7].replace(' ', '')) if row[7].replace(' ', '') != '' else 0
                autorisations_engagement_total += autorisations_engagement
                credits_paiement = int(row[9].replace(' ', '')) if row[9].replace(' ', '') != '' else 0
                credits_paiement_total += credits_paiement

                # On écrit dans le fichier cible toutes les informations utiles
                output_file.write('{};{};{};{};{};{};{};{};{};{};{};{}\n'.format(
                    type_de_mission,
                    code_type_de_mission,
                    row[0],
                    mission_to_code[row[0]],
                    code_to_programme[row[1]] if row[1] in code_to_programme else '',
                    row[1],
                    row[3],
                    row[2],
                    row[5],
                    row[4],
                    autorisations_engagement,
                    credits_paiement
                ))

print(credits_paiement_total, autorisations_engagement_total)

## 2017

In [5]:
SOURCE_FOLDER = './decoded_data/plf_2017'
TARGET_FOLDER = './parsed_data'
TARGET_FILE = 'plf_2017.csv'

type_de_mission_to_filename = {
    "Budget général": "BG",
    "Budgets annexes": "BA",
    "Comptes d'affectation spéciale": "CS",
    "Comptes de concours financiers": "CF"
}

In [6]:
mission_to_code = {}
code_to_programme = {}

with open(os.path.join('./raw_data/nomenclatures_plf/2017.csv'), 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    # On saute la première ligne parce que c'est un header
    next(csv_reader, None)

    for row in csv_reader:
        code_mission = row[3]
        mission = row[4]
        code_programme = row[5]
        programme = row[6]

        mission_to_code[mission] = code_mission
        code_to_programme[code_programme] = programme

In [7]:
credits_paiement_total = 0
autorisations_engagement_total = 0

with open(os.path.join(TARGET_FOLDER, TARGET_FILE), 'w+') as output_file:
    # Écrire les headers
    output_file.write("Type de mission;Code type de mission;Mission;Code mission;Programme;" +\
                      "Code programme;Action;Code action;Sous-action;Code sous-action;" +\
                      "Autorisations d'engagement;Crédits de paiement\n")

    # On ouvre chacun des fichiers...
    for type_de_mission in type_de_mission_to_code:
        code_type_de_mission = type_de_mission_to_code[type_de_mission]
        code_filename = type_de_mission_to_filename[type_de_mission]

        with open(os.path.join(
            SOURCE_FOLDER,
            'PLF2017-{}-Msn-Dest.csv'.format(code_filename)), 'r') as csv_file:
            
            csv_reader = csv.reader(csv_file, delimiter=';')
            # On saute les headers (4 lignes)
            for i in range(4):
                next(csv_reader, None)

            for index, row in enumerate(csv_reader):
                # Petite subtilité pour le cas 2019, certains documents csv ont trop de lignes (il y a des lignes vides à la fin)
                if (row[2] == ''):
                    continue
                    
                autorisations_engagement = int(row[7].replace(' ', '')) if row[7].replace(' ', '') != '' else 0
                autorisations_engagement_total += autorisations_engagement
                credits_paiement = int(row[9].replace(' ', '')) if row[9].replace(' ', '') != '' else 0
                credits_paiement_total += credits_paiement

                # On écrit dans le fichier cible toutes les informations utiles
                output_file.write('{};{};{};{};{};{};{};{};{};{};{};{}\n'.format(
                    type_de_mission,
                    code_type_de_mission,
                    row[0],
                    mission_to_code[row[0]],
                    code_to_programme[row[1]] if row[1] in code_to_programme else '',
                    row[1],
                    row[3],
                    row[2],
                    row[5],
                    row[4],
                    autorisations_engagement,
                    credits_paiement
                ))

print(credits_paiement_total, autorisations_engagement_total)

358102288833 361757909049


### Tests génériques

Vérifie que les lignes ont le bon nombre de colonnes.

In [13]:
def check_columns(filename):
    with open(os.path.join(TARGET_FOLDER, filename), 'r') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';')

        for index, row in enumerate(csv_reader):
            try:
                assert(len(row) == 12)
            except:
                print("Assertion error at line {}".format(index))
                print(*row)

Vérifie qu'il n'y a pas CP ou d'AE vides.

In [14]:
def check_cp_ae(filename):
    with open(os.path.join(TARGET_FOLDER, filename), 'r') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';')

        for index, row in enumerate(csv_reader):
            try:
                assert(row[10] != '' and row[11] != '')
            except:
                print("Assertion error at line {}".format(index))
                print(row[10], row[11])

##### Exécution des tests

In [15]:
TEST_FILENAMES = [
    'plf_2017.csv',
    'plf_2018.csv',
    'plf_2019.csv'
]

for filename in TEST_FILENAMES:
    check_columns(filename)
    check_cp_ae(filename)

print('PLF ok!')

PLF ok!


### Construction de l'arbre de la visualisation D3

In [18]:
def recSize(node, key):
    # Si le noeud n'est pas une feuille, on calcule sa valeur récursivement.
    if key not in node:
        node[key] = sum(list(map(lambda x: recSize(x, key), node['children'])))

    return node[key]

def build_d3_hierarchy(source_folder, source_file, target_file_name):
    with open(os.path.join(source_folder, source_file), 'r') as csv_file:
        with open(target_file_name, 'w+') as output_file:
            csv_reader = csv.reader(csv_file, delimiter=';')
            root_node = dict({
                'name': 'PLF',
                'children': []
            })

            # Skip headers
            next(csv_reader)

    #         Ici, on parcourt les lignes du CSV une par une.
    #         On consulte leur "chemain" (type de mission, mission, etc)
    #         et on ajoute ce chemin à notre json.
            for row in csv_reader:
                current_node = root_node

                type_mission = row[0]
                mission = row[2]
                programme = row[4]
                action = row[6]
                sous_action = row[8]

                nodes = [field for field in [row[0], row[2], row[4], row[6], row[8]] if field != ''] 
                for node_index, node in enumerate(nodes):
                    # On cherche si le chemin existe déjà.
                    # S'il n'existe pas, on le crée.
                    found = False
                    for index, child in enumerate(current_node['children']):
                        if child['name'] == node:
                            current_node = current_node['children'][index]
                            found = True
                            break


                    if not found:
                        current_node['children'].append(dict({
                            'name': node,
                            'code': row[2*node_index+1]
                        }))
                        current_node = current_node['children'][-1]

                        # Si le noeud n'est pas une feuille, il a des enfants...
                        if node_index != len(nodes)-1:
                            current_node['children'] = []
                        # Sinon, il a des cp et des ae.
                        else:
                            current_node['cp'] = int(row[11])
                            current_node['ae'] = int(row[10])

            # On calcule des cp et les ae des noeuds qui ne
            # sont pas des feuilles.
            recSize(root_node, 'cp')
            recSize(root_node, 'ae')

            json.dump(root_node, output_file)

#### 2019

In [5]:
SOURCE_FOLDER = './parsed_data'
SOURCE_FILE = 'plf_2019.csv'
TARGET_FILE = './parsed_data/plf_2019.json'

In [6]:
build_d3_hierarchy(SOURCE_FOLDER, SOURCE_FILE, TARGET_FILE)

#### 2018

In [8]:
SOURCE_FOLDER = './parsed_data'
SOURCE_FILE = 'plf_2018.csv'
TARGET_FILE = './parsed_data/plf_2018.json'

In [9]:
build_d3_hierarchy(SOURCE_FOLDER, SOURCE_FILE, TARGET_FILE)

#### 2017

In [19]:
SOURCE_FOLDER = './parsed_data'
SOURCE_FILE = 'plf_2017.csv'
TARGET_FILE = './parsed_data/plf_2017.json'

In [20]:
build_d3_hierarchy(SOURCE_FOLDER, SOURCE_FILE, TARGET_FILE)

```Toute chose appartient à qui sait en jouir.```